# Selección de Modelo

A continuación, se mostrará el proceso de selección del modelo, las posibles mejoras y conclusiones de la elección

In [4]:
import pandas as pd
import numpy as np
import time

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

import xgboost as xgb
from xgboost import plot_importance

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pandas'

## 0.- Carga de Conjuntos

Se cargan los .csv correspondientes al conjunto de entrenamiento y prueba. Ambos .csv contienen las carácteristicas obtenidas del notebook original. En los .csv, la primera columna corresponde a la clase de la fila.

In [ ]:
x_train = pd.read_csv('datasets/x_train.csv', header = None).to_numpy()
y_train = np.ravel(pd.read_csv('datasets/y_train.csv', header = None).to_numpy())
x_test = pd.read_csv('datasets/x_test.csv', header = None).to_numpy()
y_test = np.ravel(pd.read_csv('datasets/y_test.csv', header = None).to_numpy())

: 

In [ ]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

: 

Tambien se cargan los .csv correspondientes a los conjuntos seleccionando las características más importantes.

## 1.- Primera Evaluación

### 1.1.- Logistic Regression Evaluation

In [ ]:
logReg = LogisticRegression()
model = logReg.fit(x_train, y_train)

: 

In [ ]:
start = time.time()
y_pred = model.predict(x_test)
end = time.time()
print("Tiempo en predicción:", end - start, "[s]")

: 

In [ ]:
confusion_matrix(y_test, y_pred)

: 

In [ ]:
print(classification_report(y_test, y_pred))

: 

### 1.2.- XGBClassifier Evaluation

In [ ]:
modelxgb = xgb.XGBClassifier(random_state=1, learning_rate=0.01, objective='binary:logistic')
modelxgb = modelxgb.fit(x_train, y_train)

: 

In [ ]:
start = time.time()
y_predxgb = modelxgb.predict(x_test)
end = time.time()
print("Tiempo en predicción:", end - start, "[s]")

: 

In [ ]:
confusion_matrix(y_test, y_predxgb)

: 

In [ ]:
print(classification_report(y_test, y_predxgb))

: 

### 1.3.- Conclusión

Es posible apreciar que el modelo de Regresión Logística tiene mejor rendimiento en la clasificación que el modelo de XGBClassifier, ya que, obtiene una mejor clasificación en la clase '1' o 'Atraso' y casi el mismo rendimiento en la clase '0' o 'No Atraso'.

Además, se puede apreciar que en tiempos de procesamiento, el modelo de Regresión Logística tarda menos tiempo (0.004 [s] vs 0.03 [s]) en predecir la clase de un vuelo. Lo cual tiene relevancia a la hora de exponer el modelo y realizar multiples solicitudes en poco tiempo.

Tal como se mencionó en el notebook original, ambos rendimientos no son suficientes para resolver el problema deseado, ya que, la cantidad de falsos negativos es demasiada alta, debido al imbalance de las clases. Es por esto, que se decide evaluar el desempeño de ambos modelos aplicando un **peso** a las clases al momento de entrenar.

El modelo de Regresión Logística posee un parámetro llamado **class_weight**, que permite equiparar el peso entre las clases para mejorar el imbalance que hay en los conjuntos en el entrenamiento. Así también, existe el parámetro **scale_pos_weight** en el modelo de XGBClassifier.

## 2.- Mejoras y  Segunda Evaluación

### 2.1.- Logistic Regression Improvement

In [ ]:
logReg = LogisticRegression(class_weight = 'balanced')
model = logReg.fit(x_train, y_train)

: 

In [ ]:
start = time.time()
y_pred = model.predict(x_test)
end = time.time()
print("Tiempo en predicción:", end - start, "[s]")

: 

In [ ]:
confusion_matrix(y_test, y_pred)

: 

In [ ]:
print(classification_report(y_test, y_pred))

: 

### 2.2.- XGBClassifier Improvement

Se calcula la proporción de imbalance entre las clases.

In [ ]:
n_y0 = len(y_train[y_train == 0])
n_y1 = len(y_train[y_train == 1])
scale = n_y0/n_y1
print(scale)

: 

In [ ]:
modelxgb = xgb.XGBClassifier(random_state=1, learning_rate=0.01, objective='binary:logistic', scale_pos_weight = scale)
modelxgb = modelxgb.fit(x_train, y_train)

: 

In [ ]:
start = time.time()
y_predxgb = modelxgb.predict(x_test)
end = time.time()
print("Tiempo en predicción:", end - start, "[s]")

: 

In [ ]:
confusion_matrix(y_test, y_predxgb)

: 

In [ ]:
print(classification_report(y_test, y_predxgb))

: 

### 2.3.- Conclusión

Con los cambios realizados en ambos modelos, es posible apreciar que el rendimiento ha cambiado considerablemente. Teniendo ambos modelos un comportamiento similar, si bien la clasificación de la clase '0' o 'No Atraso' disminuyó su rendimiento (esperable por el balanceo de clases), la clasificación de la clase '1' o 'Atraso' aumento considerablemente.

En consecuencia, la cantidad de falsos negativos disminuyeron considerablemente, y por el contrario, los falsos positivos aumentaron. Pero, **¿Se puede determinar esta situación como mejor?**. Para la resolución del problema de negocio, se entenderá como positivo el aumento de falsos positivos (y por consecuencia la disminución de falsos negativos), ya que, es preferible determinar que un avión llegará atrasado y que este **no** llegue atrasado, que, determinar que un avión no llegará atrasado y si lo haga. De esta manera, se pueden tener consideraciones a la hora de que un vuelo llegue o salga del aeropuerto con un mejor resultado.

Finalmente, se considera que el mejor modelo (y el seleccionado) es el de **Regresión Logística**, el cual tiene un mejor rendimiento que el de XGBoost, y, con un tiempo de procesamiento mucho menor.

Guardar el mejor modelo en el archivo pickle_model.pkl, con la libreria de pkl.


In [ ]:
import pickle

: 

In [ ]:
with open('pickle_model.pkl','wb') as f:
    pickle.dump(model,f)
    print('dump sucess')

: 